# Sentiment Classification & How To "Frame Problems" for a Neural Network

by Andrew Trask

- **Twitter**: @iamtrask
- **Blog**: http://iamtrask.github.io

### What You Should Already Know

- neural networks, forward and back-propagation
- stochastic gradient descent
- mean squared error
- and train/test splits

### Where to Get Help if You Need it
- Re-watch previous Udacity Lectures
- Leverage the recommended Course Reading Material - [Grokking Deep Learning](https://www.manning.com/books/grokking-deep-learning) (40% Off: **traskud17**)
- Shoot me a tweet @iamtrask


### Tutorial Outline:

- Intro: The Importance of "Framing a Problem"


- Curate a Dataset
- Developing a "Predictive Theory"
- **PROJECT 1**: Quick Theory Validation


- Transforming Text to Numbers
- **PROJECT 2**: Creating the Input/Output Data


- Putting it all together in a Neural Network
- **PROJECT 3**: Building our Neural Network


- Understanding Neural Noise
- **PROJECT 4**: Making Learning Faster by Reducing Noise


- Analyzing Inefficiencies in our Network
- **PROJECT 5**: Making our Network Train and Run Faster


- Further Noise Reduction
- **PROJECT 6**: Reducing Noise by Strategically Reducing the Vocabulary


- Analysis: What's going on in the weights?

# Lesson: Curate a Dataset

In [1]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [2]:
len(reviews)

25000

In [3]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [4]:
labels[0]

'POSITIVE'

# Lesson: Develop a Predictive Theory

In [5]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


# Project 1: Quick Theory Validation

In [6]:
from collections import Counter
import numpy as np

In [8]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [9]:
for i in range(len(reviews)):
    if (labels[i] == 'POSITIVE'):
        for word in reviews[i].split(' '):
            positive_counts[word] += 1
            total_counts[word] += 1
    else:
        for word in reviews[i].split(' '):
            negative_counts[word] += 1
            total_counts[word] += 1

In [22]:
pos_neg_ratios = Counter()

for term, cnt in list(total_counts.most_common()):
    if (cnt > 300):
        pos_neg_ratios[term] = positive_counts[term] / float(negative_counts[term] + 1)
        
for word, ratio in pos_neg_ratios.most_common():
    if (ratio > 1):
        pos_neg_ratios[word] = np.log(ratio)
    else:
        pos_neg_ratios[word] = -np.log((1 / (ratio + 0.01)))

In [23]:
# words most frequently seen in a review with "POSITIVE" label
pos_neg_ratios.most_common()

[('wonderfully', 2.0218960560332353),
 ('beautifully', 1.7626953362841438),
 ('superb', 1.7091514458966952),
 ('touching', 1.637217476541176),
 ('stewart', 1.6119987332957739),
 ('wonderful', 1.5645425925262093),
 ('fantastic', 1.5048433868558566),
 ('terrific', 1.5026699370083942),
 ('noir', 1.493925025312256),
 ('outstanding', 1.4910053152089213),
 ('excellent', 1.4647538505723599),
 ('gem', 1.3932148039644643),
 ('amazing', 1.3919815802404802),
 ('kelly', 1.3461743673304654),
 ('powerful', 1.2999662776313934),
 ('remarkable', 1.2883688239495823),
 ('favorite', 1.2668956297860055),
 ('stunning', 1.2622417124499117),
 ('perfect', 1.246742480713785),
 ('favourite', 1.2411123512753928),
 ('journey', 1.2367626271489269),
 ('bond', 1.2321436812926323),
 ('brilliant', 1.2287554137664785),
 ('recommended', 1.2163953243244932),
 ('perfectly', 1.1971931173405572),
 ('emotions', 1.1753049094563641),
 ('subtle', 1.1734135017508081),
 ('rare', 1.1566438362402944),
 ('loved', 1.1563661500586044),

In [24]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(pos_neg_ratios.most_common()))[0:30]

[('waste', -2.4907515123361046),
 ('pointless', -2.3451073877136341),
 ('redeeming', -2.2667790015910296),
 ('worst', -2.1930856334332267),
 ('laughable', -2.172468615469592),
 ('awful', -2.1385076866397488),
 ('poorly', -2.1326133844207011),
 ('lame', -1.9117232884159072),
 ('horrible', -1.844894301366784),
 ('pathetic', -1.8356498920533246),
 ('wasted', -1.7773187850732606),
 ('crap', -1.7677639636718392),
 ('badly', -1.6974777654416224),
 ('worse', -1.6820086052689358),
 ('terrible', -1.6742829939664696),
 ('mess', -1.6373136300125852),
 ('garbage', -1.6342944202026248),
 ('stupid', -1.6042380193725321),
 ('dull', -1.5370486519863287),
 ('avoid', -1.5285820410306898),
 ('wooden', -1.5231262936001064),
 ('whatsoever', -1.4673642120941253),
 ('excuse', -1.4656396376364811),
 ('ridiculous', -1.4643037391798193),
 ('boring', -1.4475226133603798),
 ('bother', -1.3837974809213034),
 ('dumb', -1.3830343745326832),
 ('annoying', -1.3279306437993517),
 ('bad', -1.3181383703873577),
 ('unless